In [ ]:
import torch
import torch.nn as nn
from torch.autograd import grad
import numpy as np
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(0)


def network_dt(outputs, inputs, create_graph = True):
    return grad(outputs, inputs, torch.ones_like(outputs), create_graph=create_graph)[0]

def network_ddt(outputs, inputs, create_graph = True):

    dt = network_dt(outputs,inputs, create_graph=create_graph)
    return grad(dt, inputs, torch.ones_like(dt),create_graph=create_graph)[0]

def sample_uniform(n): 
    return torch.rand(n, 1, device=device)

class PINN(nn.Module):
    def __init__(self, width=128, depth=3, act=nn.Tanh()):
        super().__init__()
        layers = []
        for i in range(depth):
            layers += [nn.Linear(2 if i==0 else width, width), act]
        layers += [nn.Linear(width, 1)]
        self.net = nn.Sequential(*layers)
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)
    def forward(self, x, t):
        return self.net(torch.cat([x, t], dim=1))

model = PINN().to(device)

def compute_PDE_loss(net,x,t,N,L):
    dx = L/N
    y = net(x,t)
    yddx = (net(x+dx,t)-2*y+net(x-dx,t))/dx**2
    ydt = grad(y,t,torch.ones_like(y),retain_graph=True,create_graph=True)[0]
    yddt = grad(ydt,t,torch.ones_like(y),retain_graph=True,create_graph=True)[0]
    residual = yddt - yddx
    return residual

L = np.pi
N = 100

x = torch.tensor([[1.0]], device=device)
t = torch.tensor([[0.25]], device=device,requires_grad=True)

compute_PDE_loss(model,x,t,N,L)
# test = model.forward(x,t)


# # choose counts
# N_u, N_f = 10, 100

# # boundary/initial (examples; targets added in Step 4)
# x_b0 = torch.zeros(N_u//2, 1, device=device); t_b0 = sample_uniform(N_u//2)  # x=0
# x_b1 = torch.ones (N_u//2, 1, device=device); t_b1 = sample_uniform(N_u//2)  # x=1
# x_i  = sample_uniform(N_u);                 t_i0 = torch.zeros_like(x_i)      # t=0

# # interior collocation
# x_f = sample_uniform(N_f); t_f = sample_uniform(N_f)








tensor([[0.1221]], grad_fn=<SubBackward0>)

In [9]:
# wave_pinn_fixed.py
import torch
import torch.nn as nn
from torch.autograd import grad
import math

# ----- Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(0)
torch.set_default_dtype(torch.float32)

# ----- Derivative helper
def d(outputs, inputs, create_graph=True):
    """First derivative: ∂outputs/∂inputs via vector–Jacobian product with ones."""
    return grad(outputs, inputs, torch.ones_like(outputs), create_graph=create_graph)[0]

# ----- Network
class PINN(nn.Module):
    def __init__(self, in_dim=2, width=128, depth=4, act=nn.Tanh()):
        super().__init__()
        layers = []
        for i in range(depth):
            layers += [nn.Linear(in_dim if i == 0 else width, width), act]
        layers += [nn.Linear(width, 1)]
        self.net = nn.Sequential(*layers)
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x, t):
        z = torch.cat([x, t], dim=1)  # (N,2)
        return self.net(z)

# ----- Problem setup
c = 1.0
N_f = 10_000   # collocation points
N_b = 1_000    # boundary points
N_i = 1_000    # initial line points

rand = lambda n: torch.rand(n, 1, device=device)

# Interior points (we differentiate w.r.t. these each step)
x_f = rand(N_f)
t_f = rand(N_f)

# Boundary points (no derivatives needed)
t_b = rand(N_b)
x_b0 = torch.zeros_like(t_b, device=device)
x_b1 = torch.ones_like(t_b, device=device)

# Initial line t=0 (we differentiate wrt t here)
x_i = rand(N_i)
t_i0 = torch.zeros_like(x_i, device=device)

# Ground-truth functions (for IC & evaluation)
def u_true(x, t):
    return torch.sin(math.pi * x) * torch.cos(math.pi * t)

def ut_true(x, t):
    return -math.pi * torch.sin(math.pi * x) * torch.sin(math.pi * t)

# IC targets (detach to avoid any graph ties)
with torch.no_grad():
    u0 = u_true(x_i, torch.zeros_like(x_i))
    v0 = ut_true(x_i, torch.zeros_like(x_i))  # equals 0 for this choice

# ----- Model & optimizer
model = PINN().to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
mse = lambda x: torch.mean(x**2)

# ----- PDE residual
def pde_residual(net, x, t):
    u    = net(x, t)       # (N,1)
    u_t  = d(u, t)         # ∂u/∂t
    u_tt = d(u_t, t)       # ∂²u/∂t²
    u_x  = d(u, x)         # ∂u/∂x
    u_xx = d(u_x, x)       # ∂²u/∂x²
    return u_tt - (c**2) * u_xx

# ----- Training
steps = 500
for step in range(1, steps + 1):
    # Fresh leaves for tensors we differentiate w.r.t. this step
    x_f = x_f.detach().requires_grad_(True)
    t_f = t_f.detach().requires_grad_(True)
    x_i = x_i.detach().requires_grad_(True)
    t_i0 = t_i0.detach().requires_grad_(True)

    # PDE loss
    r = pde_residual(model, x_f, t_f)
    L_pde = mse(r)

    # Boundary (Dirichlet u=0 at x=0 and x=1) — no grads w.r.t. coords needed
    u_b0 = model(x_b0, t_b)
    u_b1 = model(x_b1, t_b)
    L_bc = mse(u_b0) + mse(u_b1)

    # Initial conditions at t=0: u(x,0)=sin(pi x), u_t(x,0)=0
    u_i  = model(x_i, t_i0)
    u_ti = d(u_i, t_i0)  # ∂u/∂t at t=0
    # u0, v0 are constants (no grad)
    L_ic = mse(u_i - u0) + mse(u_ti - v0)

    loss = L_pde + L_bc + L_ic

    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()

    if step % 10 == 0:
        print(f"step {step:5d} | L_pde {L_pde.item():.3e} | L_bc {L_bc.item():.3e} | L_ic {L_ic.item():.3e} | total {loss.item():.3e}")

# ----- Evaluation
with torch.no_grad():
    nx, nt = 200, 200
    xg = torch.linspace(0, 1, nx, device=device).view(-1, 1)
    tg = torch.linspace(0, 1, nt, device=device).view(-1, 1)
    # default indexing in torch.meshgrid is fine here
    Xg, Tg = torch.meshgrid(xg.squeeze(1), tg.squeeze(1))
    x_flat = Xg.reshape(-1, 1)
    t_flat = Tg.reshape(-1, 1)

    up = model(x_flat, t_flat).reshape(nx, nt)
    ut = u_true(Xg, Tg)

    num = torch.linalg.norm((up - ut).reshape(-1))
    den = torch.linalg.norm(ut.reshape(-1))
    rel_l2 = (num / den).item()
    print(f"\nRelative L2 error on grid: {rel_l2:.3e}")


step    10 | L_pde 4.376e-05 | L_bc 3.826e-02 | L_ic 3.067e-01 | total 3.450e-01
step    20 | L_pde 1.886e-04 | L_bc 8.827e-02 | L_ic 2.396e-01 | total 3.280e-01
step    30 | L_pde 1.880e-04 | L_bc 7.512e-02 | L_ic 2.492e-01 | total 3.245e-01
step    40 | L_pde 1.359e-03 | L_bc 6.622e-02 | L_ic 2.518e-01 | total 3.193e-01
step    50 | L_pde 5.356e-04 | L_bc 7.694e-02 | L_ic 2.282e-01 | total 3.057e-01
step    60 | L_pde 1.832e-03 | L_bc 7.356e-02 | L_ic 2.125e-01 | total 2.879e-01
step    70 | L_pde 1.186e-03 | L_bc 6.405e-02 | L_ic 2.155e-01 | total 2.807e-01
step    80 | L_pde 3.735e-03 | L_bc 6.722e-02 | L_ic 1.931e-01 | total 2.640e-01
step    90 | L_pde 2.314e-02 | L_bc 6.665e-02 | L_ic 1.504e-01 | total 2.401e-01
step   100 | L_pde 1.711e-02 | L_bc 5.059e-02 | L_ic 1.388e-01 | total 2.065e-01
step   110 | L_pde 1.275e-02 | L_bc 5.673e-02 | L_ic 8.721e-02 | total 1.567e-01
step   120 | L_pde 1.457e-02 | L_bc 4.695e-02 | L_ic 5.338e-02 | total 1.149e-01
step   130 | L_pde 1.409e-02

c:\Users\jakob\anaconda3\envs\thesis\Lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4324.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
